# Anomalies zonales et transitoires

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Les fichiers de données au format netcdf (moyennes quotidiennes NCEP) doivent être récupérés par FTP (FileZilla) et placés dans le répertoire data :

Hauteur géopotentielle : ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/pressure/hgt.XXXX.nc

Il faut télécharger au moins la période 2000-2005 et ensuite concaténer les fichier netcdf avec les commandes NCO ncrcat ou CDO mergetime :

- https://linux.die.net/man/1/ncrcat
- https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-930002.2.6

--> fichier z.nc

<div class="alert alert-warning">
    
Moyenne temporelle : $$ \overline{z} = \frac{1}{T} \int_0^T z dt $$

Moyenne zonale : $$ \left[ z \right] = \frac{1}{2\pi} \int_0^{2\pi} z d\lambda $$

Anomalie temporelle : $$ z^\prime = z - \overline{z} $$

Anomalie zonale : $$ z^* = z - \left[ z \right] $$
    
</div>

<div class="alert alert-warning">

$$z=\overline{[z]}+\overline{z^*}+[z]'+z^{*'}$$

Composante axisymétrique et stationnaire de la circulation : $$\overline{[z]}$$
Ecart stationnaire à la moyenne zonale : $$\overline{z^*}$$
Perturbation transitoire de la moyenne zonale : $$[z]'$$
Perturbation transitoire non axisymétrique : $$z^{*'}$$
    
</div>

In [ ]:
%matplotlib inline

import os

import xarray as xr
import numpy as np
from netCDF4 import Dataset
import h5py

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

def plot_zonal_mean2(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 10)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(da[lon_name] > 180, da[ lon_name] - 360, da[lon_name])
    da = (
        da .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

proj=ccrs.EqualEarth()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

# Lecture des données

In [ ]:
year1='2015'
year2='2020'

In [ ]:
diri="./daily_ncep/"

fz    = xr.open_dataset(diri+"z.nc").sel(time=slice(year1,year2))

lat  = fz.lat.values
lev = fz.level.values

print(fz)
print(lev)

z=fz['hgt']
z=lonflip(z)

lon = z.lon.values

# Fonctions pour les moyennes et les anomalies

Moyenne temporelle : $$ \overline{z} = \frac{1}{T} \int_0^T z dt $$

Moyenne zonale : $$ \left[ z \right] = \frac{1}{2\pi} \int_0^{2\pi} z d\lambda $$

Anomalie temporelle : $$ z^\prime = z - \overline{z} $$

Anomalie zonale : $$ z^* = z - \left[ z \right] $$

In [ ]:
def bar(data, interval=fz.time.dt.dayofyear):
    return data.groupby(interval).mean(skipna=True)

def bracket(data):
    return data.mean(dim='lon', skipna=True)

def prime(data, interval=fz.time.dt.dayofyear):
    return data.groupby(interval) - bar(data, interval=interval)

def star(data):
    return data - bracket(data)

In [ ]:
zbar=bar(z)
zprime=prime(z)
z_zmean=bracket(z)
zstar=star(z)

print(zbar.shape)
print(zprime.shape)
print(z_zmean.shape)
print(zstar.shape)

In [ ]:
z, z_zmean, = xr.broadcast(z,z_zmean)

print(z_zmean.shape)

In [ ]:
date='2015-01-01'
level=500

In [ ]:
z_day = z.sel(time=date).sel(level=level)
zclim_day = zbar.sel(dayofyear=1).sel(level=level)
z_zmean_day = z_zmean.sel(time=date).sel(level=level)
zprime_day = zprime.sel(time=date).sel(level=level)
zstar_day = zstar.sel(time=date).sel(level=level)

In [ ]:
print(z_day.shape)
print(zclim_day.shape)
print(z_zmean_day.shape)
print(zprime_day.shape)
print(zstar_day.shape)

In [ ]:
levels_z = np.arange(4800,6050,50)
levels_zanom = np.arange(-500,550,50)


fig = plt.figure(figsize=(19,7))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(231, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, z_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(232, projection=proj)
plot_background(ax)
ax.set_title("$[z]$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmean_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_zmean_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(233, projection=proj)
plot_background(ax)
ax.set_title("$z^* = z-[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zstar_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstar_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(234, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, z_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(235, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zclim_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zclim_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(236, projection=proj)
plot_background(ax)
ax.set_title("$z' = z-\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zprime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zprime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


figname='./figs/z'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, z_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z0_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$[z]$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmean_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_zmean_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z1_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z^* = z-[z]$", fontsize=14)
cf = ax.contourf(lon, lat, zstar_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstar_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z2_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zclim_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, zclim_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z3_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z' = z-\overline{z}$", fontsize=14)
cf = ax.contourf(lon, lat, zprime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zprime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z4_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-danger">

    
<b>Coder la décomposition suivante : </b>
    
$$z=\overline{[z]}+\overline{z^*}+[z]'+z^{*'}$$

Composante axisymétrique et stationnaire de la circulation : $$\overline{[z]}$$
Ecart stationnaire à la moyenne zonale : $$\overline{z^*}$$
Perturbation transitoire de la moyenne zonale : $$[z]'$$
Perturbation transitoire non axisymétrique : $$z^{*'}$$

</div>

In [ ]:
z_zmeanbar=bar(bracket(z))
zstarbar=bar(star(z))
z_zmeanprime=prime(bracket(z))
zstarprime=prime(star(z))

In [ ]:
print(z_zmeanbar.shape)
print(zstarbar.shape)
print(z_zmeanprime.shape)
print(zstarprime.shape)

In [ ]:
zstarprime, z_zmeanprime, = xr.broadcast(zstarprime,z_zmeanprime)
zstarbar, z_zmeanbar, = xr.broadcast(zstarbar,z_zmeanbar)
print(z_zmeanbar.shape)
print(z_zmeanprime.shape)

In [ ]:
date='2015-01-01'
level=500

In [ ]:
z_day = z.sel(time=date).sel(level=level)

zstarbar_day = zstarbar.sel(dayofyear=1).sel(level=level)
z_zmeanbar_day = z_zmeanbar.sel(dayofyear=1).sel(level=level)
z_zmean_prime_day =  z_zmeanprime.sel(time=date).sel(level=level)
zstar_prime_day = zstarprime.sel(time=date).sel(level=level)

sum=zstarbar_day+z_zmeanbar_day+z_zmean_prime_day+zstar_prime_day

print(zclim_day.shape)

In [ ]:
levels_z = np.arange(4800,6050,50)
levels_zanom2 = np.arange(-250,270,20)
levels_zanom2 = np.arange(-250,270,20)

fig = plt.figure(figsize=(19,7))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(231, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, z_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(232, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmeanbar_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_zmeanbar_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(233, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z^*}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarbar_day, levels_zanom2, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarbar_day, levels_zanom2, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

ax=fig.add_subplot(234, projection=proj)
plot_background(ax)
ax.set_title("$[z]'$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmean_prime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, z_zmean_prime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(235, projection=proj)
plot_background(ax)
ax.set_title("$z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, zstar_prime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstar_prime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


ax=fig.add_subplot(236, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}+\overline{z^*}+[z]'+z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, sum, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, sum, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')


figname='./figs/z_decomp_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("z", fontsize=14)
cf = ax.contourf(lon, lat, z_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp0_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{[z]}$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmeanbar_day, levels_z, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
c = ax.contour(lon, lat, z_zmeanbar_day, levels_z, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp1_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$\overline{z^*}$", fontsize=14)
cf = ax.contourf(lon, lat, zstarbar_day, levels_zanom2, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstarbar_day, levels_zanom2, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp2_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$[z]'$", fontsize=14)
cf = ax.contourf(lon, lat, z_zmean_prime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, z_zmean_prime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp3_'+date
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.suptitle('Geopotential height at '+str(level)+' hPa : '+date, fontsize=16)

ax=fig.add_subplot(111, projection=proj)
plot_background(ax)
ax.set_title("$z^{*'}$", fontsize=14)
cf = ax.contourf(lon, lat, zstar_prime_day, levels_zanom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
c = ax.contour(lon, lat, zstar_prime_day, levels_zanom, colors= 'black', linewidths=0.2, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
cb.set_label('m', size='small')

figname='./figs/z_decomp4_'+date
fig.savefig(figname+'.png',bbox_inches='tight')